In [ ]:
%pip install pandas
%pip install openpyxl

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, when, sum as spark_sum, avg, lit,regexp_extract

In [17]:
spark = SparkSession.builder.config("spark.driver.host", "localhost").config("spark.driver.memory", "4g").config("spark.executor.memory", "4g").appName("SalestoSQLLite").getOrCreate()

In [3]:
def read_excel_to_spark(file_path, region):
    # Read Excel file using pandas
    df_pandas = pd.read_excel(file_path, engine='openpyxl')  # You can add the sheet name if needed
    
    # Convert pandas DataFrame to PySpark DataFrame
    df_spark = spark.createDataFrame(df_pandas)
    
    # Add the region column to identify the source
    df_spark = df_spark.withColumn("region", lit(region))
    
    return df_spark

In [4]:
df_region_a = read_excel_to_spark("order_region_a.xlsx", "A")
df_region_b = read_excel_to_spark("order_region_b.xlsx", "B")

In [5]:
df_sales = df_region_a.union(df_region_b)

In [6]:
df_sales.show()

+-------------------+--------------+---------------+---------+--------------------+--------+------+
|            OrderId|   OrderItemId|QuantityOrdered|ItemPrice|   PromotionDiscount|batch_id|region|
+-------------------+--------------+---------------+---------+--------------------+--------+------+
|171-0001135-1657958|11168926687715|              1|    949.0|{ "CurrencyCode":...|     359|     A|
|171-0001497-9165123|19760298917699|              1|    699.0|{ "CurrencyCode":...|    1135|     A|
|171-0002127-1363507| 5949764099083|              1|    399.0|{ "CurrencyCode":...|     297|     A|
|171-0002370-0601169|57571868836379|              1|    499.0|{ "CurrencyCode":...|     114|     A|
|171-0004526-2028348|33851287891403|              1|   1699.0|{ "CurrencyCode":...|     764|     A|
|171-0004781-3853173|43686103544491|              1|    399.0|{ "CurrencyCode":...|     809|     A|
|171-0004947-4305927|15941372058555|              1|   1399.0|{ "CurrencyCode":...|      15|     A|


In [7]:
df_sales = df_sales.withColumn(
    "PromotionDiscount", 
    regexp_extract(col("PromotionDiscount"), r'"Amount":\s*"([\d.]+)"', 1).cast("float")
)
df_sales.show()

+-------------------+--------------+---------------+---------+-----------------+--------+------+
|            OrderId|   OrderItemId|QuantityOrdered|ItemPrice|PromotionDiscount|batch_id|region|
+-------------------+--------------+---------------+---------+-----------------+--------+------+
|171-0001135-1657958|11168926687715|              1|    949.0|             10.0|     359|     A|
|171-0001497-9165123|19760298917699|              1|    699.0|             10.1|    1135|     A|
|171-0002127-1363507| 5949764099083|              1|    399.0|             10.0|     297|     A|
|171-0002370-0601169|57571868836379|              1|    499.0|             10.1|     114|     A|
|171-0004526-2028348|33851287891403|              1|   1699.0|             10.0|     764|     A|
|171-0004781-3853173|43686103544491|              1|    399.0|             10.1|     809|     A|
|171-0004947-4305927|15941372058555|              1|   1399.0|             10.0|      15|     A|
|171-0004947-4305927|159413720

In [22]:
df_sales = df_sales.withColumn("total_sales", col("QuantityOrdered") * col("ItemPrice"))
df_sales.show()

+-------------------+--------------+---------------+---------+-----------------+--------+------+-----------+
|            OrderId|   OrderItemId|QuantityOrdered|ItemPrice|PromotionDiscount|batch_id|region|total_sales|
+-------------------+--------------+---------------+---------+-----------------+--------+------+-----------+
|171-0001135-1657958|11168926687715|              1|    949.0|             10.0|     359|     A|      949.0|
|171-0001497-9165123|19760298917699|              1|    699.0|             10.1|    1135|     A|      699.0|
|171-0002127-1363507| 5949764099083|              1|    399.0|             10.0|     297|     A|      399.0|
|171-0002370-0601169|57571868836379|              1|    499.0|             10.1|     114|     A|      499.0|
|171-0004526-2028348|33851287891403|              1|   1699.0|             10.0|     764|     A|     1699.0|
|171-0004781-3853173|43686103544491|              1|    399.0|             10.1|     809|     A|      399.0|
|171-0004947-430592

In [24]:
df_sales = df_sales.dropDuplicates()
df_sales.show()

Py4JJavaError: An error occurred while calling o267.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8.0 failed 1 times, most recent failure: Lost task 1.0 in stage 8.0 (TID 40) (SunilsThinkpad executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scal

In [21]:
duplicate_records = df_sales.groupBy("OrderId").count().filter("count > 1")

# Show the duplicate records
duplicate_records.show(1)

Py4JJavaError: An error occurred while calling o258.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 6.0 failed 1 times, most recent failure: Lost task 4.0 in stage 6.0 (TID 33) (SunilsThinkpad executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scal

In [18]:
driver_memory = spark.sparkContext.getConf().get("spark.driver.memory")
print(driver_memory)

None


In [ ]:
df_sales = df_sales.withColumn("net_sale", col("total_sales") - col("PromotionDiscount"))

In [ ]:
df_sales = df_sales.filter(col("net_sale") > 0)

In [ ]:
df_sales.show()

In [ ]:
import sys
print(sys.executable)